In [ ]:
import torchimport torch.nn as nnimport torch.optim as optimimport torchvisionimport torchvision.transforms as transformsimport matplotlib.pyplot as pltdevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")num_epochs = 5batch_size = 64learning_rate = 0.01momentum = 0.9transform = transforms.Compose([    transforms.ToTensor(),    transforms.Normalize((0.5,), (0.5,))])train_dataset = torchvision.datasets.FashionMNIST(root='./data',                                                  train=True,                                                  transform=transform,                                                  download=True)test_dataset = torchvision.datasets.FashionMNIST(root='./data',                                                 train=False,                                                 transform=transform)train_loader = torch.utils.data.DataLoader(dataset=train_dataset,                                           batch_size=batch_size,                                           shuffle=True)test_loader = torch.utils.data.DataLoader(dataset=test_dataset,                                          batch_size=batch_size,                                          shuffle=False)classes = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']class CNN(nn.Module):    def __init__(self):        super(CNN, self).__init__()        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2)        self.pool = nn.MaxPool2d(2, 2)        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)        self.fc1 = nn.Linear(32 * 7 * 7, 128)        self.fc2 = nn.Linear(128, 10)        self.relu = nn.ReLU()    def forward(self, x):        x = self.relu(self.conv1(x))        x = self.pool(x)        x = self.relu(self.conv2(x))        x = self.pool(x)        x = x.view(-1, 32 * 7 * 7)        x = self.relu(self.fc1(x))        x = self.fc2(x)        return xmodel = CNN().to(device)criterion = nn.CrossEntropyLoss()optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, nesterov=True)total_step = len(train_loader)for epoch in range(num_epochs):    for i, (images, labels) in enumerate(train_loader):        images = images.to(device)        labels = labels.to(device)        outputs = model(images)        loss = criterion(outputs, labels)        optimizer.zero_grad()        loss.backward()        optimizer.step()        if (i+1) % 100 == 0:            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}")model.eval()with torch.no_grad():    correct = 0    total = 0    for images, labels in test_loader:        images = images.to(device)        labels = labels.to(device)        outputs = model(images)        _, predicted = torch.max(outputs.data, 1)        total += labels.size(0)        correct += (predicted == labels).sum().item()    print(f"Test Accuracy: {100 * correct / total:.2f}%")